In [ ]:
import sys
import os
import IPython
os.environ['NUMBA_CACHE_DIR'] = IPython.paths.get_ipython_cache_dir() # librosa 충돌 방지
sys.path.append(os.path.join(os.path.dirname(""), "..")) # custom import 용도
import custom
import numpy as np 

from transformers import AutoProcessor, AutoModelForCTC # hugging face 전처리 함수, 신경망 함수
import json # 음절-라벨 사전
import librosa # 소리 파일 불러오기

import torch # torch.tensor 형태로 전처리, 신경망에서 출력 받아서 데이터 처리

In [ ]:
# 전처리 함수, 신경망 함수 불러오기
load_directory = './STT/'

model = AutoModelForCTC.from_pretrained(load_directory)
tokenizer = AutoProcessor.from_pretrained(load_directory)

In [ ]:
# 음절-라벨 사전 불러오기
with open("STT/vocab.json", mode = "r") as f:
    sound_dict = json.load(f)
    num_dict = {v : k for k, v in sound_dict.items()}

print(len(sound_dict))

1452


In [ ]:
# 테스트용 소리 파일 불러오기

file = "test.wav"

sig, freq = librosa.load(file, sr = 16000)
print(sig.shape)
print(sig.dtype)
print(freq)
print(sig.min(), sig.max())

(52302,)
float32
16000
-0.38356403 0.33580145


In [ ]:
# hugging face 전처리 함수

data = tokenizer(sig, sampling_rate=freq, return_tensors="pt").input_values
print(data)
print(data.shape)

tensor([[-0.0003, -0.0003, -0.0003,  ..., -0.0003, -0.0003, -0.0003]])
torch.Size([1, 52302])


In [ ]:
# hugging face 신경망 함수

y = model(data).logits
print(y)
print(y.shape)

tensor([[[  2.7693,  -0.6829,  -5.8537,  ...,  16.0733, -11.0533, -11.0285],
         [  2.5907,  -0.8442,  -5.6037,  ...,  19.4212, -12.8144, -12.8727],
         [  2.7735,  -0.9018,  -5.6587,  ...,  20.0292, -13.3239, -13.3753],
         ...,
         [  0.8001,  -1.4136,  -6.5083,  ...,  21.9192, -13.6953, -13.8740],
         [  5.0338,  -0.8160,  -7.1029,  ...,  22.9838, -15.2330, -15.4004],
         [ 20.9327,   0.9146,  -2.1321,  ...,   7.1836,  -5.4667,  -5.3573]]],
       grad_fn=<ViewBackward0>)
torch.Size([1, 163, 1454])


In [ ]:
# 결과값 해석

argmax = torch.argmax(y, dim = -1).squeeze().detach().numpy().tolist() # torch data를 list로 변환

text = custom.word_vectorize(argmax, num_dict)
text = "".join(text).replace("[PAD]","").replace("|"," ")

print(text)

디법프 
